In [4]:
import os, sys
sys.path.append(os.path.join(os.path.dirname(__file__), "cultural_artifacts"))

NameError: name '__file__' is not defined

In [1]:
from src.opensource.llama_frontend import chat_pipeline

ModuleNotFoundError: No module named 'torch'

In [ ]:
agent = chat_pipeline()

In [ ]:
inst_template = """
    [INST] Name the object on the basis of the above clues. After your guess, I will let you know if you are correct or not. If you're correct, do not make any further guesses and end the conversation. If you are wrong, make your second and final guess. I do not need to know your reasoning behind the answer. Just tell me the answer. If you do not know the answer, say that you do not know the answer.[/INST]
"""

In [ ]:
output_1 = agent.predict(human_input=inst_template)

In [ ]:
output_1

In [ ]:
output_2 = agent.predict(human_input="Your first guess is correct")

In [ ]:
output_2